In [3]:
import pandas as pd
import numpy as np
import os

# ---------------------------
# CARGA DE ARCHIVO
# ---------------------------
addresses = [
    'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/FURANOS.xlsx',
    'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/FURANOS.xlsx'
]

df = None
for path in addresses:
    if os.path.exists(path):
        df = pd.read_excel(path, header=1)
        print(f"✅ Archivo cargado desde: {path}")
        break

if df is None:
    raise FileNotFoundError("❌ No se encontró el archivo en ninguna de las rutas especificadas.")
df["SERIE"] = df["SERIE"].astype(str)
df['SERIE'] = df['SERIE'].astype(str).str.replace(" ", "")

✅ Archivo cargado desde: C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/FURANOS.xlsx


In [4]:
# ---------------------------
# LIMPIEZA DE DATOS
# ---------------------------
# df["SERIE"] = df["SERIE"].astype(str)
df = df[['SERIE', 'Fecha', '2-Furfuraldehido (FAL, ppb)']]
# df = df.dropna(axis=1, how='all')
df['Fecha'] = pd.to_datetime(df['Fecha'], errors="coerce")

# ---------------------------
# CONFIGURACIÓN DE PUNTAJE FUR
# ---------------------------
# Verificar si la columna existe y tiene datos
if '2-Furfuraldehido (FAL, ppb)' in df.columns:
    df['FUR'] = np.select(
        [
            df['2-Furfuraldehido (FAL, ppb)'] > 5000,
            (df['2-Furfuraldehido (FAL, ppb)'] > 1000) & (df['2-Furfuraldehido (FAL, ppb)'] <= 5000),
            (df['2-Furfuraldehido (FAL, ppb)'] > 500) & (df['2-Furfuraldehido (FAL, ppb)'] <= 1000),
            (df['2-Furfuraldehido (FAL, ppb)'] > 100) & (df['2-Furfuraldehido (FAL, ppb)'] <= 500),
            df['2-Furfuraldehido (FAL, ppb)'] <= 100
        ],
        [5, 4, 3, 2, 1],
        default=np.nan
    )
else:
    df['FUR'] = np.nan
df.head()

,SERIE,Fecha,"2-Furfuraldehido (FAL, ppb)",FUR
0,D518293,2023-08-29,1.0,1.0
1,D518292,2023-08-29,1.0,1.0
2,D518291,2023-08-29,1.0,1.0
3,V101038,2023-09-03,1.0,1.0
4,551873,2023-12-05,1.0,1.0


In [3]:
# ---------------------------
# TABLAS BASE
# ---------------------------
df = df.rename(columns={'Fecha': 'FECHA'})
df_FUR = df[['SERIE', 'FECHA', 'FUR']].copy()

# ---------------------------
# EXTENSIÓN DEL CALENDARIO DESDE 2015
# ---------------------------
inicio = "2015-01-01"
fecha_inicio = pd.Timestamp(inicio)   # en el 2026 cambiar si hace falta
fecha_fin = pd.Timestamp.today().normalize()
fechas = pd.date_range(fecha_inicio, fecha_fin, freq="D")

todas_series = df['SERIE'].dropna().unique()
calendario = pd.MultiIndex.from_product([todas_series, fechas], names=["SERIE", "FECHA"])
df_calendario = pd.DataFrame(index=calendario).reset_index()

# ---------- Tabla extendida FUR ----------
ultimos_2024 = df_FUR[df_FUR['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024['FECHA'] = fecha_inicio
base_ext = pd.concat([df_FUR, ultimos_2024], ignore_index=True)

df_extendida_FUR = pd.merge(df_calendario, base_ext, on=["SERIE","FECHA"], how="left")
df_extendida_FUR = df_extendida_FUR.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

# ---------- Tabla de detalles originales (datos + FUR) ----------
df_detalles_FUR = df.copy()   # ya contiene FUR

# ---------- Tabla de detalles extendida ----------
ultimos_2024_det = df[df['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024_det['FECHA'] = fecha_inicio
base_ext_det = pd.concat([df, ultimos_2024_det], ignore_index=True)

df_extendida_detalles_FUR = pd.merge(df_calendario, base_ext_det, on=["SERIE","FECHA"], how="left")
df_extendida_detalles_FUR = df_extendida_detalles_FUR.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

C:\Users\roquispec\AppData\Local\Temp\ipykernel_22656\200850986.py:25: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_extendida_FUR = df_extendida_FUR.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
C:\Users\roquispec\AppData\Local\Temp\ipykernel_22656\200850986.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_extendida_detalles_FUR = df_extendida_detalles_FUR.groupby("SERIE").apply(lambda

In [4]:
# esta ya tiene FUR, no necesitas mergear otra vez
df_detalles_ext_FUR = df_extendida_detalles_FUR.copy()

# ---------------------------
# FUNCIONES PARA LLAMAR
# ---------------------------
def get_df_FUR():
    return df_FUR

def get_df_extendida_FUR():
    return df_extendida_FUR

def get_df_detalles_FUR():
    return df_detalles_FUR

def get_df_detalles_ext_FUR():
    return df_detalles_ext_FUR

# ---------------------------
# DEMO DE RESULTADOS
# ---------------------------
print("\n ====== TABLA CON FECHAS ORIGINALES ====== \n")
print(get_df_FUR(), "\n")

print("\n ====== TABLA CON FECHAS EXTENDIDAS ====== \n")
print(get_df_extendida_FUR(), "\n")

print("\n ====== TABLA DE DETALLES DE FUR CON FECHAS ORIGINALES ====== \n")
print(get_df_detalles_FUR(), "\n")

print("\n ====== TABLA DE DETALLES DE FUR CON FECHAS EXTENDIDAS ====== \n")
print(get_df_detalles_ext_FUR(), "\n")


 ====== TABLA CON FECHAS ORIGINALES ====== 

         SERIE      FECHA  FUR
0      D518293 2023-08-29  1.0
1      D518292 2023-08-29  1.0
2      D518291 2023-08-29  1.0
3      V101038 2023-09-03  1.0
4       551873 2023-12-05  1.0
..         ...        ...  ...
128   132006T1 2020-01-01  1.0
129    123158T 2023-07-18  1.0
130    L-30436 2023-10-23  1.0
131    123160T 2009-06-02  4.0
132  750005-01 2018-08-02  1.0

[133 rows x 3 columns] 


 ====== TABLA CON FECHAS EXTENDIDAS ====== 

          SERIE      FECHA  FUR
0        100138 2015-01-01  NaN
1        100138 2015-01-02  NaN
2        100138 2015-01-03  NaN
3        100138 2015-01-04  NaN
4        100138 2015-01-05  NaN
...         ...        ...  ...
526675  V180403 2025-10-30  1.0
526676  V180403 2025-10-31  1.0
526677  V180403 2025-11-01  1.0
526678  V180403 2025-11-02  1.0
526679  V180403 2025-11-03  1.0

[526680 rows x 3 columns] 


 ====== TABLA DE DETALLES DE FUR CON FECHAS ORIGINALES ====== 

         SERIE      FECHA  2-Fur